In [1]:
import os
import sys
import torchvision
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1125 06:44:28.695613 140462858368832 file_utils.py:39] PyTorch version 1.1.0 available.
I1125 06:44:29.029454 140462858368832 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, WarmupLinearSchedule
from tqdm import tqdm_notebook as tqdm

I1125 06:44:29.582342 140462858368832 corenlp.py:42] Using an existing server http://140.109.19.191:9000
I1125 06:44:30.585761 140462858368832 corenlp.py:118] The server is available.


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_set = FgcSerDataset(config.FGC_TRAIN,
                        transform=torchvision.transforms.Compose([BertIdx(tokenizer)]))

I1125 06:44:31.506117 140462858368832 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


preprocessing /work/fgc_support_retri/data/FGC/1.2/FGC_release_A_train(cn).json ......


Expecting value: line 1 column 1 (char 0)
D036
Expecting value: line 1 column 1 (char 0)
D040
aspan error:  2019 
{'QTYPE': '基础题', 'AMODE': 'Multi-Spans-Extraction', 'ASPAN': [{'text': '木雕博馆广场', 'end': 36, 'start': 30}, {'text': '水美商圈', 'end': 41, 'start': 37}, {'text': ' 2019', 'end': 4, 'start': -1}, {'text': '三义木雕艺术节', 'end': 11, 'start': 4}, {'text': '擧行', 'end': 45, 'start': 43}], 'ATYPE': 'Location', 'SHINT': [{'text': '2019三义木雕艺术节系列活动将从10月10日起一连四天，在木雕博馆广场及水美商圈盛大擧行，现场除了国内外的木雕作品展示，还有艺文表演、茶席体验等，民众不妨趁著双十假期到三义感受慢城之美', 'end': 93, 'start': 0}, {'text': ' 记者\u3000陈怡伶\u3000报导 2019三义木雕艺术节系列活动以「艺动-木艺慢活嘉年华」为主题，结合三义慢城意象，以及独特木雕与茶文化特色，邀请民众来一趟深度探索之旅，三义木雕博物馆也配合这次活动，自国庆日起连续四天免费开放参观', 'end': 203, 'start': 94}], 'QTEXT': '2019三义木雕艺术节活动于三义的哪两个地点举行?', 'QID': 'D068Q05', 'ANSWER': [{'ATOKEN': [{'text': '木雕博馆广场', 'start': 30}, {'text': '水美商圈', 'start': 37}], 'ATEXT': '木雕博馆广场及水美商圈'}]}

data size = 332


In [5]:
len(train_set)

3150

train model

In [6]:
torch.manual_seed(12)
bert_model_name = 'bert-base-chinese'
warmup_proportion = 0.1
learning_rate = 5e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_num = 0 if torch.cuda.is_available() else -1
n_gpu = torch.cuda.device_count()

bert_encoder = BertModel.from_pretrained(bert_model_name)
model = BertSupSentClassification(bert_encoder)



model.to(device)
if n_gpu > 1:
    model = nn.DataParallel(model)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

print('start training ... ')

model.train()
print('done')

I1125 06:46:59.585567 140462858368832 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1125 06:46:59.587941 140462858368832 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "fi

start training ... 
done


In [7]:
batch_size = 32
dataloader_train = DataLoader(train_set, batch_size=batch_size)

for epoch_i in range(10):

    running_loss = 0.0
    for batch_i, batch in enumerate(tqdm(dataloader_train)):
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        loss = model(input_ids, token_type_ids=token_type_ids,
                     attention_mask=attention_mask, mode=BertSupSentClassification.ForwardMode.TRAIN,
                     labels=labels)

        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu.

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_i % 10 == 0:
            print('[%d, %5d] loss: %.3f' %
                  (epoch_i + 1, batch_i + 1, running_loss / 10))
            running_loss = 0.0

/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


[1,     1] loss: 0.065
[1,    11] loss: 0.558
[1,    21] loss: 0.467
[1,    31] loss: 0.406
[1,    41] loss: 0.316
[1,    51] loss: 0.370
[1,    61] loss: 0.251
[1,    71] loss: 0.396
[1,    81] loss: 0.389
[1,    91] loss: 0.532



/work/fgc_support_retri/fgc_support_retri/fgc_preprocess.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample['label'] = torch.tensor(sample['label'], dtype=torch.long)


[2,     1] loss: 0.042
[2,    11] loss: 0.507
[2,    21] loss: 0.371
[2,    31] loss: 0.342
[2,    41] loss: 0.318
[2,    51] loss: 0.344
[2,    61] loss: 0.234
[2,    71] loss: 0.315
[2,    81] loss: 0.340
[2,    91] loss: 0.412



[3,     1] loss: 0.041
[3,    11] loss: 0.463
[3,    21] loss: 0.336
[3,    31] loss: 0.270
[3,    41] loss: 0.301
[3,    51] loss: 0.350
[3,    61] loss: 0.230
[3,    71] loss: 0.321
[3,    81] loss: 0.339
[3,    91] loss: 0.415



...

[6,     1] loss: 0.040
[6,    11] loss: 0.392
[6,    21] loss: 0.279
[6,    31] loss: 0.241
[6,    41] loss: 0.230
[6,    51] loss: 0.234
[6,    61] loss: 0.185
[6,    71] loss: 0.230
[6,    81] loss: 0.251
[6,    91] loss: 0.267



[7,     1] loss: 0.023
[7,    11] loss: 0.416
[7,    21] loss: 0.258
[7,    31] loss: 0.235
[7,    41] loss: 0.225
[7,    51] loss: 0.309
[7,    61] loss: 0.201
[7,    71] loss: 0.206
[7,    81] loss: 0.218
[7,    91] loss: 0.285



[8,     1] loss: 0.029
[8,    11] loss: 0.497
[8,    21] loss: 0.285
[8,    31] loss: 0.239
[8,    41] loss: 0.223
[8,    51] loss: 0.250
[8,    61] loss: 0.173
[8,    71] loss: 0.229
[8,    81] loss: 0.206
[8,    91] loss: 0.234



[9,     1] loss: 0.030
[9,    11] loss: 0.360
[9,    21] loss: 0.262
[9,    31] loss: 0.207
[9,    41] loss: 0.198
[9,    51] loss: 0.238
[9,    61] loss: 0.172
[9,    71] loss: 0.210
[9,    81] loss: 0.193
[9,    91] loss: 0.219



[10,     1] loss: 0.021
[10,    11] loss: 0.348
[10,    21] loss: 0.211
[10,    31] loss: 0.205
[10,    41] loss: 0.200
[10,    51] loss: 0.255
[10,    61] loss: 0.172
[10,    71] loss: 0.225
[10,    81] loss: 0.203
[10,    91] loss: 0.216



In [8]:
model_to_save = model.module if hasattr(model, 'module') else model
torch.save(model_to_save.state_dict(), 'trainedmodel.m')

In [9]:
torch.cuda.empty_cache()

In [11]:
del model